# Evaluate and re-run OD calls

Once you have completed the first run of the OD analysis, come here to evaluate holes in the dataset and the ru-run missing values

In [1]:
import os, sys, importlib

import pandas as pd
import numpy as np

In [2]:
xx = pd.read_csv('J:/Data/PROJECTS/COUNTRY/SVK/european__OD.csv')
origins = pd.read_csv('J:/Data/PROJECTS/COUNTRY/SVK/origin_grid.csv')
destinations = pd.read_csv('J:/Data/PROJECTS/COUNTRY/SVK/european_destinations.csv')

In [3]:
yy = xx.groupby("O_UID")
bad_origins = np.where(yy['D_UID'].count() < 127)

In [ ]:
def get_missing_destinations(inD):
    dests = list(inD['D_UID'])
    dOIDs = [str(x) for x in range(1, 128) if not x in dests]
    return("_".join(dOIDs))

res = yy.apply(get_missing_destinations)

In [ ]:
baseFolder = 'C:/temp/SVK/RUN2'
count = 0
for unq in res.unique():
    if unq != '':
        selFeatures = res[res == unq]
        badDests = [int(X) for X in unq.split("_")]
        badOIDs = selFeatures.index
        badO = origins.loc[list(badOIDs)]
        badD = destinations[destinations['ID'].isin(badDests)]
        outFolder = "%s/bad_%s" % (baseFolder, count)
        if not os.path.exists(outFolder):
            os.makedirs(outFolder)
        badO.to_csv(os.path.join(outFolder, "origins.csv"))
        badD.to_csv(os.path.join(outFolder, "destinations.csv"))
        print(r"python OD.py -od -s {baseFolder}\bad_{n}\origins.csv -d {baseFolder}\bad_{n}\destinations.csv --lon Long --lat Lat -outputOD {baseFolder}\bad_{n}\od.csv".format(n=count, baseFolder=baseFolder))
        count += 1

In [5]:
# Process the re-processed values and re-write the output
baseFolder = 'C:/temp/SVK'
allOD = []
for root, dirs, files in os.walk(baseFolder):
    for f in files:
        if f == "od.csv":
            allOD.append(os.path.join(root, f))

In [7]:
try:
    del(final)
except:
    pass
for x in allOD:
    cRes = pd.read_csv(x)
    try:
        final = final.append(cRes)
    except:
        final = cRes

In [8]:
print(final.shape)
print(xx.shape)

(378192, 6)
(1180352, 6)


In [9]:
xx_combined = xx.append(final)

In [15]:
xx.shape


(1180352, 6)

In [16]:
xx_combined.to_csv('J:/Data/PROJECTS/COUNTRY/SVK/european__OD_Corrected2.csv')